In [1]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.11.0


In [2]:
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.utils import to_categorical
from keras.layers import MaxPooling2D, Conv2D
import numpy as np
import os
from tensorflow.python.keras.callbacks import EarlyStopping
from datetime import datetime

##  Load frames

In [6]:
drowsines_levels = np.array(["0", "5", "10"])
DATA_PATH = os.path.join('./Dataset')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for video_index in range(1,11):
  for level in drowsines_levels:
    window = []      
    for frame in range(1, 4001, 2):
      file_path = os.path.join(DATA_PATH, str(video_index), level, (str(frame) + ".npy"))
      if os.path.exists(file_path):
        res = np.load(file_path)        
        window.append(res)
      else:
        print(f"File at {file_path} doesn't exists")
    files.append(window)
    labels.append(label_map[level])
  print(f"File number: {video_index} processed")

File number: 1 processed
File number: 2 processed
File number: 3 processed
File number: 4 processed
File number: 5 processed
File number: 6 processed
File number: 7 processed
File number: 8 processed
File number: 9 processed
File number: 10 processed


In [7]:
# Check loaded files shape
np.array(files).shape

(30, 2000, 1434)

## Model training

In [ ]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs_{date_string}')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', kernel_regularizer='l2' , input_shape=(2000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=500, batch_size=500, shuffle = True, callbacks=[tb_callback], validation_split = 0.2) 

model.save(f'drowsines_levels_weights_${date_string}.h5')

## Model validation

### Confusion Matrix

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

multilabel_confusion_matrix(ytrue, yhat)

### Tensorboard